RAW DATA PRE-PROCESSING

In [1]:
'''
Read raw biomarker data from csv, flatten by collection date
re-arrange the biomarkers in suitable date: day 1 = admission date
mark conflicting, missing data for manual correction
'''
import numpy as np
import pandas as pd

data_path = "../../data/"
raw_data = data_path + "updated_rawdata_2024_v22.xlsx"
orig_data = data_path + "All CENTER TRAJ data (to share) 6-3-19 v2.xlsx"
tony_462 = data_path + "462_samples_with_adm_date_updated_v2.csv"

validation_results = '../../results/validation/new_data02.csv'
validation_missing_data = '../../results/validation/new_missing_data1.csv'

In [2]:
rawdf = pd.read_excel(raw_data, sheet_name="Sheet1", header=0)
orgdf = pd.read_excel(orig_data, sheet_name="adjusted", header=0)
tdf = pd.read_csv(tony_462, header = 0)

d:\Applications\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
target_col_name = tdf.columns.tolist()
target_data_type = tdf.dtypes

In [4]:
new_col_name = ['extra', 'Id', 'NSE', 'Date','Sample', 'S100B', 'GFAP','Tau', 'UCH-L1', 'NFL']
biomarkers = ['NSE', 'S100B', 'GFAP','Tau', 'UCH-L1', 'NFL']

df = rawdf[rawdf['subjectId'].isin(tdf['subjectId_x'])]

subjects = df['subjectId'].unique()
df.columns = new_col_name
df.loc[:, 'extra'] = None

dftest = df[df.loc[:, 'Id'] == '2AAC636']
print(dftest)
min_idx = dftest['Date'].dropna().idxmin()
print(dftest.loc[min_idx, 'NSE'])
fst_day = dftest.loc[min_idx, 'Date'] + pd.Timedelta(days=2)
if dftest['Date'].isin([fst_day]).any():
    nx_idx = dftest['Date'].index[dftest['Date'] == fst_day]
    print(dftest.loc[nx_idx, 'GFAP'])
else:
    print('Not found')



     extra       Id     NSE       Date      Sample  S100B  GFAP     Tau  \
1184  None  2AAC636  156.60        NaT  BIOM049633  3.420  1.41  133.30   
1185  None  2AAC636   17.54 1970-01-16  BIOM094097  0.040  0.25    1.21   
1186  None  2AAC636   33.89 1970-01-03  BIOM094105  0.177  2.55    3.76   
1187  None  2AAC636   27.54 1970-01-05  BIOM094113  0.179  0.45    1.38   

       UCH-L1     NFL  
1184  1332.75   33.02  
1185   101.06  610.90  
1186   140.28   64.16  
1187    57.71  119.17  
33.89
1187    0.45
Name: GFAP, dtype: float64


In [35]:
value_map = {
    'Collection_date_'  : 'Date',
    'sampleid_'         : 'Sample',
    'NSE_'              : 'NSE',
    'S100B_'            : 'S100B',
    'GFAP_'             : 'GFAP',
    'Tau_'              : 'Tau',
    'UCH-L1_'           : 'UCH-L1',
    'NFL_'              : 'NFL'
}
dfx = pd.DataFrame(columns=target_data_type.index)
for i, id in enumerate(subjects):
    dfi = df[df.loc[:, 'Id'] == id]
    # print('dfi = \n', dfi)
    nday = dfi['Date'].dropna().min()
    # row = {}
    dfx.loc[i, 'subjectId_x'] = id
    counter = 0
    for j in range(min(5, len(dfi))):
        if counter > 4:
            break
        
        if dfi['Date'].isin([nday]).any():
            # idx_day = dfi['Date'].isin([nday]).index[0]
            idx_days = dfi[dfi['Date'] == nday].index
            if (len(idx_days) > 1):
                dfx.loc[i, 'Manual_Review'] = 'B'
            for ix, idx_day in enumerate(idx_days):
                col_suffix = str(counter + 1)
                for k, v in value_map.items():
                    dfx.loc[i, k + col_suffix] = dfi.loc[idx_day, v]
                counter += 1
                if counter > 4:
                    break
        else:
            counter += 1
        nday = nday + pd.Timedelta(days=1)                        
            
        
            
    if (counter < 5 and dfi['Date'].isnull().any()):
        dfx.loc[i, 'Manual_Review'] = 'A'
print(dfx.shape)

    

(461, 46)


In [17]:
print(dfx)

    subjectId_x Hospital.ICUAdmDate Subject.PatientType  \
0       6yeM346                 NaN                 NaN   
1       9Ydj395                 NaN                 NaN   
2       9Khz638                 NaN                 NaN   
3       6ban548                 NaN                 NaN   
4       4uVH383                 NaN                 NaN   
..          ...                 ...                 ...   
456     1kAg181                 NaN                 NaN   
457     8AZV086                 NaN                 NaN   
458     8jAn024                 NaN                 NaN   
459     5BwA829                 NaN                 NaN   
460     5btz622                 NaN                 NaN   

    Subject.DateOfAdmission    Collection_date_1    Collection_date_2  \
0                       NaN  1970-01-01 00:00:00  1970-01-02 00:00:00   
1                       NaN  1970-01-01 00:00:00  1970-01-02 00:00:00   
2                       NaN  1970-01-02 00:00:00  1970-01-03 00:00:00   

In [80]:
# Export to file for manual correction
print(dfx)

    subjectId_x Hospital.ICUAdmDate Subject.PatientType  \
0       6yeM346                 NaN                 NaN   
1       9Ydj395                 NaN                 NaN   
2       9Khz638                 NaN                 NaN   
3       6ban548                 NaN                 NaN   
4       4uVH383                 NaN                 NaN   
..          ...                 ...                 ...   
456     1kAg181                 NaN                 NaN   
457     8AZV086                 NaN                 NaN   
458     8jAn024                 NaN                 NaN   
459     5BwA829                 NaN                 NaN   
460     5btz622                 NaN                 NaN   

    Subject.DateOfAdmission Collection_date_1 Collection_date_2  \
0                       NaN        1970-01-01        1970-01-02   
1                       NaN        1970-01-01        1970-01-02   
2                       NaN        1970-01-02        1970-01-03   
3                      

In [36]:
dfx.to_csv(validation_results, header=True, index=False)

In [88]:
# Data that need to update manually selected 18 rows
missing_data_id = dfx[dfx['Manual_Review'] == 'A']['subjectId_x']
dfy = df[df['Id'].isin(missing_data_id)]
dfy = dfy[dfy['Date'].isnull()]
dfy.to_csv(validation_missing_data, header=True, index=False)
print(dfy)

     extra       Id     NSE Date      Sample     S100B    GFAP     Tau  \
0     None  6yeM346   10.29  NaT  BIOM006825     0.012    1.18    1.60   
673   None  8Epx355   17.12  NaT  BIOM094185     0.388    0.36    3.04   
1065  None  7keL575   32.95  NaT  BIOM064913     0.448  129.05   62.23   
1068  None  7keL575   49.41  NaT  BIOM065009     1.570  131.22   84.03   
1184  None  2AAC636  156.60  NaT  BIOM049633     3.420    1.41  133.30   
2504  None  5nrb942   16.13  NaT  BIOM047337     0.159   13.34   10.98   
3868  None  3xSE269   13.88  NaT  BIOM048257     0.214   42.24    4.89   
4417  None  5hki863   46.09  NaT  BIOM046049     0.750   27.05   74.63   
6870  None  3TjZ823   10.17  NaT  BIOM030849     0.030    0.24    0.91   
7438  None  3gMW929   19.36  NaT  BIOM017953     0.255   23.47    9.14   
7595  None  5Hem626   27.59  NaT  BIOM008273     0.412   38.09   10.62   
7607  None  8cmk298   11.16  NaT  BIOM017721     0.040    0.18    0.85   
7609  None  8cmk298   15.30  NaT  BIOM